In [2]:
import os

import logs_analyzer as analyzer
import matplotlib.pyplot as plt
from os.path import join
from os import listdir
from pathlib import Path
import tarfile
import shutil

plt.style.use({'figure.facecolor':'white'})

In [3]:
PROTOCOLS = {"bracha", "scalable", "witness"}
COMPRESSED = "compressed"
LOG_DIR = "logs"

ns = [16, 32, 64, 96, 128, 160, 192, 224, 256]

In [4]:
def get_stat(mode="stress"):
    statistics = {}

    for directory in listdir(LOG_DIR):
        if not isdir(join(LOG_DIR, directory)):
            continue

        ind = directory.find(mode)
        if ind == -1:
            continue

        if mode == "config":
            splitted_dir = directory[ind:].split("_")
            thr = int(splitted_dir[1])
            params_str = directory[:ind]
        else:
            thr = directory[ind:]
            params_str = directory[:ind]

        params = params_str.split("Input")
        protocol = params[0]
        n = ns[int(params[1])]

        print(directory)

        if statistics.get(n) is None:
            statistics[n] = {}
        if statistics[n].get(protocol) is None:
            statistics[n][protocol] = {}

        statistics[n][protocol][thr] = \
            analyzer.calculate_stat(
                directory=join(LOG_DIR, directory, "outputs"),
                n=n
            )

    return statistics

In [5]:
config_statistics = get_stat(mode="config")

FileNotFoundError: [Errno 2] No such file or directory: 'logs'

In [ ]:
PROTOCOL_TO_COLOR = {
    "bracha": "blue",
    "scalable": "red",
    "witness": "green"
}

def plot(protocol, data):
    xs = list(data.keys())
    xs.sort()
    ys = [data[x] for x in xs]

    color = PROTOCOL_TO_COLOR[protocol]
    plt.scatter(x=xs, y=ys, color=color)
    label = protocol
    if protocol == "bracha":
        label = "Bracha"
    plt.plot(xs, ys, color=color, label=label)

In [ ]:
def build_thr_to_latency(statistics):
    throughput_to_latency = {}

    for protocol in PROTOCOLS:
        throughput_to_latency[protocol] = {}
        for n in ns:
            if statistics.get(n) is None or statistics[n].get(protocol) is None:
                continue

            data = []
            for rate, stat in statistics[n][protocol].items():
                if rate >= 21 or stat["transaction_cnt"] == 0:
                    continue

                data.append((
                    stat["avg_throughput"],
                    stat["avg_transaction_latency"]
                ))
            data = sorted(data, key=lambda pr: pr[0])

            adjusted_data = {}
            i = 0
            while i < len(data):
                j = i
                sum_latency = 0
                while j < len(data) and data[j][0] == data[i][0]:
                    sum_latency += data[j][1]
                    j += 1

                adjusted_data[data[i][0]] = sum_latency / (j - i)

                i = j
            throughput_to_latency[protocol][n] = adjusted_data

    return throughput_to_latency

In [ ]:
config_throughput_to_latency = build_thr_to_latency(config_statistics)
stress_throughput_to_latency = build_thr_to_latency(stress_statistics)

In [ ]:
def build_latency_throughput_graph(throughput_to_latency, n):
    for protocol in PROTOCOLS:
        if throughput_to_latency.get(protocol) is None \
                or throughput_to_latency[protocol].get(n) is None:
            continue

        data = throughput_to_latency[protocol][n]
        plot(protocol=protocol, data=data)

    plt.xlabel(xlabel="throughput per second")
    plt.ylabel(ylabel="latency, s")
    plt.legend()

    plt.savefig(f"graphs/latency_throughput/{n}.png")
    plt.show()

In [ ]:
build_latency_throughput_graph(config_throughput_to_latency, 32)

In [ ]:
metrics_to_label = {
    "avg_transaction_latency": "Latency, s",
    "avg_throughput": "Throughput"
}

In [ ]:
def build_rate_to_metrics(
        n,
        metrics="avg_transaction_latency",
        is_normalized=False
):
    rate_to_metrics = {}

    for protocol in PROTOCOLS:
        rate_to_metrics[protocol] = {}

    for protocol, protocol_stat in config_statistics[n].items():
        curr_rate_to_latency = {}
        for rate, stat in protocol_stat.items():
            if rate >= 21 or stat["transaction_cnt"] == 0:
                continue
            curr_rate_to_latency[rate] = stat[metrics]

        rate_sorted = list(curr_rate_to_latency.keys())
        rate_sorted.sort()

        for curr_rate in rate_sorted:
            rate_to_metrics[protocol][curr_rate] = curr_rate_to_latency[curr_rate]

    if is_normalized:
        for rate in rate_to_metrics["bracha"].keys():
            for protocol in PROTOCOLS:
                if protocol == "bracha":
                    continue
                rate_to_metrics[protocol][rate] /= rate_to_metrics["bracha"][rate]
            rate_to_metrics["bracha"][rate] = 1.0

    for protocol in PROTOCOLS:
        plot(protocol, rate_to_metrics[protocol])

    plt.xlabel(xlabel="Rate of transactions / 1s")
    plt.ylabel(ylabel=metrics_to_label[metrics])
    plt.legend()

    parameter = metrics.split("_")[-1]
    output_dir = "normalized" if is_normalized else "absolute"
    plt.savefig(f"graphs/rate_{parameter}/{output_dir}/{n}.png")
    plt.show()

In [ ]:
for metrics in "avg_transaction_latency", "avg_throughput":
    for is_normalized in True, False:
        for n in ns:
            build_rate_to_metrics(
                n=n,
                metrics=metrics,
                is_normalized=is_normalized
            )

In [ ]:
def build_n_metrics(
        rate,
        metrics="avg_transaction_latency",
        is_normalized=False
):
    protocol_to_stat = {}
    for protocol in PROTOCOLS:
        protocol_to_stat[protocol] = {}

    for n, n_stat in config_statistics.items():
        for protocol, protocol_stat in n_stat.items():
            protocol_to_stat[protocol][n] = float(protocol_stat[rate][metrics])

    if is_normalized:
        for n in ns:
            for protocol in PROTOCOLS:
                if protocol == "bracha":
                    continue
                protocol_to_stat[protocol][n] /= protocol_to_stat["bracha"][n]

            protocol_to_stat["bracha"][n] = 1.0

    for protocol in PROTOCOLS:
        plot(protocol, protocol_to_stat[protocol])

    plt.xlabel(xlabel="Number of processes")
    plt.ylabel(ylabel=metrics_to_label[metrics])
    plt.legend()

    parameter = metrics.split("_")[-1]
    output_dir = "normalized" if is_normalized else "absolute"
    output_path = f"graphs/n_{parameter}/{output_dir}"
    Path(output_path).mkdir(parents=True, exist_ok=True)

    plt.savefig(f"{output_path}/{rate}.png")
    plt.show()

In [ ]:
build_n_metrics(
    rate=16,
    metrics="avg_transaction_latency",
    is_normalized=True
)

In [ ]:
for metrics in "avg_transaction_latency", "avg_throughput":
    for is_normalized in True, False:
        build_n_metrics(
            rate=16,
            metrics=metrics,
            is_normalized=is_normalized
        )